## Subsetting the NETCDF4 data by Specifying Lat/Lon Range

The ERA5 dataset we used came from two sources:
1. Sinica: level-based data with domain of 10 \~ 50N/100 \~ 140E (161x161, 0.25 degree interval)
2. NTU: MSLP data with domain of -20 \~ 60N/60 \~ 180E (321x481, 0.25 degree interval)

In order to make further analysis easier, we need to subset the MSLP data to fit the domain of level-based data. We refer to [this article](https://stackoverflow.com/questions/48683135/how-to-subset-a-netcdf-file-using-latlong-details-and-rewrite-to-a-new-nc-file) to do the job.


### Test Ground for Subsetting

In [1]:
# Development test
from netCDF4 import Dataset
import numpy as np

lon0 = 100.
lon1 = 140.
lat0 = 10.
lat1 = 50.


nch500 = Dataset('../data/era5/h500/2018123100_h500.nc','r')
ncmslp = Dataset('D:\data\ERA5_nc\MSLP\MSLP_20181231.nc')

print(nch500)
print(ncmslp)

lons = ncmslp.variables['lon'][:]
lats = ncmslp.variables['lat'][:]

lat_lb = np.argmin(abs(lats-lat0))
lat_ub = np.argmin(abs(lats-lat1))+1

lon_lb = np.argmin(abs(lons-lon0))
lon_ub = np.argmin(abs(lons-lon1))+1

print(lat_lb, lat_ub, lon_lb, lon_ub)
print(lats[lat_lb], lats[lat_ub], lons[lon_lb], lons[lon_ub])

mslp = ncmslp.variables['psl'][0, lon_lb:lon_ub, lat_lb:lat_ub]
print(mslp.shape)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2021-03-30 15:42:51 GMT by grib_to_netcdf-2.18.0: grib_to_netcdf -o ../../../data/ERA5_nc//2018123100_h500.nc tmp.grb
    dimensions(sizes): longitude(161), latitude(161), time(1)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), int16 z(time, latitude, longitude)
    groups: 
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    institution: European Centre for Medium-Range Weather Forecasts
    history: 2021-04-10T17:07:51 GRIB to CDM+CF via cfgrib-0.9.8.4/ecCodes-2.16.0 with {"source": "/cache/data0/2b4cb15ef928098cd31ecd2f6b1e26c7.grib", "filter_by_keys": {}, "encode_cf": ["parameter", "time", "geography", "vertical"]}
    source: ECMWF
    dimensions(sizes): time(1), lat(321), lon(481)
    variables(dimensions): int64 time(time), int

### Function for single file conversion

In [2]:
def subset_era5_nc(infile, outfile, lat0=10., lat1=50., lon0=100., lon1=140., var='psl'):
    from netCDF4 import Dataset
    import numpy as np
    import time
    # Read input
    nc_file = Dataset(infile,'r')
    # Retrieve lons/lats
    lats = nc_file.variables['lat'][:]
    lons = nc_file.variables['lon'][:]
    # Calculat bounds
    lat_lb = np.argmin(abs(lats-lat0))
    lat_ub = np.argmin(abs(lats-lat1))+1
    lon_lb = np.argmin(abs(lons-lon0))
    lon_ub = np.argmin(abs(lons-lon1))+1
    # Subset the MLSP and lon/lat
    var_sub = nc_file.variables[var][0, lon_lb:lon_ub, lat_lb:lat_ub]
    var_units = nc_file.variables[var].units
    lat_sub = nc_file.variables['lat'][lat_lb:lat_ub]
    lon_sub = nc_file.variables['lon'][lon_lb:lon_ub]
    nc_file.close()

    # write to new file
    my_file = Dataset(outfile,'w', format = 'NETCDF4')
    my_file.discription = 'ERA5-MSLP-subset'
    my_file.history = 'Created on: ' +time.ctime(time.time())
    # Dimensions
    latdim = lat_ub-lat_lb # getting the no of element
    londim = lon_ub-lon_lb
    lon = my_file.createDimension('longitude',londim)
    lat = my_file.createDimension('lattitude',latdim)
    t = my_file.createDimension('time',1)
    # Variables
    latitude = my_file.createVariable('Lattitude',np.float32,('lattitude',))
    latitude.units = 'Degree_north'
    longitude = my_file.createVariable('Longitude',np.float32,('longitude',))
    longitude.units = 'Degree_east'
    ts = my_file.createVariable('time',np.int32,('time',))
    var = my_file.createVariable('slp',np.float32,('time','longitude','lattitude'),fill_value = -9999.0)
    var.units = var_units
    #Load values to the variables
    latitude[:] = lat_sub
    longitude[:] = lon_sub
    var[:] = var_sub
    my_file.close()
    
# Test the function
subset_era5_nc('D:\data\ERA5_nc\MSLP\MSLP_20181231.nc', '../data/era5/mslp/2018123100_mslp.nc')

In [3]:
# See test results
test = Dataset('../data/era5/mslp/2018123100_mslp.nc')
print(test)
print(test.variables['Lattitude'][:])
print(test.variables['Longitude'][:])
print(test.variables['slp'][:])

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    discription: ERA5-MSLP-subset
    history: Created on: Tue Apr 20 17:33:40 2021
    dimensions(sizes): longitude(161), lattitude(161), time(1)
    variables(dimensions): float32 Lattitude(lattitude), float32 Longitude(longitude), int32 time(time), float32 slp(time, longitude, lattitude)
    groups: 
[10.   10.25 10.5  10.75 11.   11.25 11.5  11.75 12.   12.25 12.5  12.75
 13.   13.25 13.5  13.75 14.   14.25 14.5  14.75 15.   15.25 15.5  15.75
 16.   16.25 16.5  16.75 17.   17.25 17.5  17.75 18.   18.25 18.5  18.75
 19.   19.25 19.5  19.75 20.   20.25 20.5  20.75 21.   21.25 21.5  21.75
 22.   22.25 22.5  22.75 23.   23.25 23.5  23.75 24.   24.25 24.5  24.75
 25.   25.25 25.5  25.75 26.   26.25 26.5  26.75 27.   27.25 27.5  27.75
 28.   28.25 28.5  28.75 29.   29.25 29.5  29.75 30.   30.25 30.5  30.75
 31.   31.25 31.5  31.75 32.   32.25 32.5  32.75 33.   33.25 33.5  33.75
 34.   34.25 34.5  34.75 

### File collecting and Parsing

In [6]:
def list_era5_files(dir, prefix='MSLP_', suffix='.nc'):
    ''' To scan through the sapecified dir and get the corresponding file with suffix. '''
    import os
    import pandas as pd
    xfiles = []
    for root, dirs, files in os.walk(dir, followlinks=True):    # Loop through the directory
        for fn in files:
            if fn.endswith(suffix):                             # Filter files with suffix
                timestamp = fn.replace(suffix,'').replace(prefix,'')          # Removing the suffix to get time-stamp
                xfiles.append({'timestamp':timestamp, 'xuri':os.path.join(root, fn)})
    return(pd.DataFrame(xfiles).sort_values('timestamp').reset_index(drop=True))

# Test
infiles = list_era5_files('D:\data\ERA5_nc\MSLP')
print(infiles.head())

  timestamp                                   xuri
0  19790101  D:\data\ERA5_nc\MSLP\MSLP_19790101.nc
1  19790102  D:\data\ERA5_nc\MSLP\MSLP_19790102.nc
2  19790103  D:\data\ERA5_nc\MSLP\MSLP_19790103.nc
3  19790104  D:\data\ERA5_nc\MSLP\MSLP_19790104.nc
4  19790105  D:\data\ERA5_nc\MSLP\MSLP_19790105.nc


### Batch Processing

In [9]:
SRCPATH = 'D:\data\ERA5_nc\MSLP'
OUTPATH = '../data/era5/mslp/'
OUT_SUFFIX = '_mslp.nc'

#for i in range(100,110,2):
for i in range(infiles.shape[0]):
    infile = infiles.iloc[i]['xuri']
    outfile = OUTPATH + infiles.iloc[i]['timestamp'] + OUT_SUFFIX
    print(infile, outfile)
    subset_era5_nc(infile, outfile)
    

D:\data\ERA5_nc\MSLP\MSLP_19790101.nc ../data/era5/mslp/19790101_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790102.nc ../data/era5/mslp/19790102_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790103.nc ../data/era5/mslp/19790103_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790104.nc ../data/era5/mslp/19790104_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790105.nc ../data/era5/mslp/19790105_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790106.nc ../data/era5/mslp/19790106_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790107.nc ../data/era5/mslp/19790107_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790108.nc ../data/era5/mslp/19790108_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790109.nc ../data/era5/mslp/19790109_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790110.nc ../data/era5/mslp/19790110_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790111.nc ../data/era5/mslp/19790111_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790112.nc ../data/era5/mslp/19790112_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790113.nc ../data/era5/mslp/19790113_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790114.nc ../data/era5/

D:\data\ERA5_nc\MSLP\MSLP_19790505.nc ../data/era5/mslp/19790505_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790506.nc ../data/era5/mslp/19790506_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790507.nc ../data/era5/mslp/19790507_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790508.nc ../data/era5/mslp/19790508_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790509.nc ../data/era5/mslp/19790509_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790510.nc ../data/era5/mslp/19790510_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790511.nc ../data/era5/mslp/19790511_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790512.nc ../data/era5/mslp/19790512_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790513.nc ../data/era5/mslp/19790513_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790514.nc ../data/era5/mslp/19790514_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790515.nc ../data/era5/mslp/19790515_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790516.nc ../data/era5/mslp/19790516_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790517.nc ../data/era5/mslp/19790517_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790518.nc ../data/era5/

D:\data\ERA5_nc\MSLP\MSLP_19790928.nc ../data/era5/mslp/19790928_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790929.nc ../data/era5/mslp/19790929_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19790930.nc ../data/era5/mslp/19790930_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791001.nc ../data/era5/mslp/19791001_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791002.nc ../data/era5/mslp/19791002_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791003.nc ../data/era5/mslp/19791003_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791004.nc ../data/era5/mslp/19791004_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791005.nc ../data/era5/mslp/19791005_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791006.nc ../data/era5/mslp/19791006_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791007.nc ../data/era5/mslp/19791007_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791008.nc ../data/era5/mslp/19791008_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791009.nc ../data/era5/mslp/19791009_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791010.nc ../data/era5/mslp/19791010_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19791011.nc ../data/era5/

D:\data\ERA5_nc\MSLP\MSLP_19800218.nc ../data/era5/mslp/19800218_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800219.nc ../data/era5/mslp/19800219_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800220.nc ../data/era5/mslp/19800220_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800221.nc ../data/era5/mslp/19800221_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800222.nc ../data/era5/mslp/19800222_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800223.nc ../data/era5/mslp/19800223_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800224.nc ../data/era5/mslp/19800224_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800225.nc ../data/era5/mslp/19800225_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800226.nc ../data/era5/mslp/19800226_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800227.nc ../data/era5/mslp/19800227_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800228.nc ../data/era5/mslp/19800228_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800229.nc ../data/era5/mslp/19800229_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800301.nc ../data/era5/mslp/19800301_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800302.nc ../data/era5/

D:\data\ERA5_nc\MSLP\MSLP_19800620.nc ../data/era5/mslp/19800620_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800621.nc ../data/era5/mslp/19800621_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800622.nc ../data/era5/mslp/19800622_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800623.nc ../data/era5/mslp/19800623_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800624.nc ../data/era5/mslp/19800624_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800625.nc ../data/era5/mslp/19800625_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800626.nc ../data/era5/mslp/19800626_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800627.nc ../data/era5/mslp/19800627_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800628.nc ../data/era5/mslp/19800628_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800629.nc ../data/era5/mslp/19800629_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800630.nc ../data/era5/mslp/19800630_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800701.nc ../data/era5/mslp/19800701_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800702.nc ../data/era5/mslp/19800702_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_19800703.nc ../data/era5/

...

D:\data\ERA5_nc\MSLP\MSLP_20180912.nc ../data/era5/mslp/20180912_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180913.nc ../data/era5/mslp/20180913_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180914.nc ../data/era5/mslp/20180914_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180915.nc ../data/era5/mslp/20180915_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180916.nc ../data/era5/mslp/20180916_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180917.nc ../data/era5/mslp/20180917_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180918.nc ../data/era5/mslp/20180918_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180919.nc ../data/era5/mslp/20180919_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180920.nc ../data/era5/mslp/20180920_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180921.nc ../data/era5/mslp/20180921_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180922.nc ../data/era5/mslp/20180922_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180923.nc ../data/era5/mslp/20180923_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180924.nc ../data/era5/mslp/20180924_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20180925.nc ../data/era5/

D:\data\ERA5_nc\MSLP\MSLP_20190103.nc ../data/era5/mslp/20190103_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190104.nc ../data/era5/mslp/20190104_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190105.nc ../data/era5/mslp/20190105_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190106.nc ../data/era5/mslp/20190106_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190107.nc ../data/era5/mslp/20190107_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190108.nc ../data/era5/mslp/20190108_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190109.nc ../data/era5/mslp/20190109_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190110.nc ../data/era5/mslp/20190110_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190111.nc ../data/era5/mslp/20190111_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190112.nc ../data/era5/mslp/20190112_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190113.nc ../data/era5/mslp/20190113_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190114.nc ../data/era5/mslp/20190114_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190115.nc ../data/era5/mslp/20190115_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190116.nc ../data/era5/

D:\data\ERA5_nc\MSLP\MSLP_20190427.nc ../data/era5/mslp/20190427_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190428.nc ../data/era5/mslp/20190428_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190429.nc ../data/era5/mslp/20190429_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190430.nc ../data/era5/mslp/20190430_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190501.nc ../data/era5/mslp/20190501_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190502.nc ../data/era5/mslp/20190502_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190503.nc ../data/era5/mslp/20190503_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190504.nc ../data/era5/mslp/20190504_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190505.nc ../data/era5/mslp/20190505_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190506.nc ../data/era5/mslp/20190506_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190507.nc ../data/era5/mslp/20190507_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190508.nc ../data/era5/mslp/20190508_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190509.nc ../data/era5/mslp/20190509_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190510.nc ../data/era5/

D:\data\ERA5_nc\MSLP\MSLP_20190821.nc ../data/era5/mslp/20190821_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190822.nc ../data/era5/mslp/20190822_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190823.nc ../data/era5/mslp/20190823_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190824.nc ../data/era5/mslp/20190824_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190825.nc ../data/era5/mslp/20190825_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190826.nc ../data/era5/mslp/20190826_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190827.nc ../data/era5/mslp/20190827_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190828.nc ../data/era5/mslp/20190828_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190829.nc ../data/era5/mslp/20190829_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190830.nc ../data/era5/mslp/20190830_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190831.nc ../data/era5/mslp/20190831_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190901.nc ../data/era5/mslp/20190901_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190902.nc ../data/era5/mslp/20190902_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20190903.nc ../data/era5/

D:\data\ERA5_nc\MSLP\MSLP_20191225.nc ../data/era5/mslp/20191225_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20191226.nc ../data/era5/mslp/20191226_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20191227.nc ../data/era5/mslp/20191227_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20191228.nc ../data/era5/mslp/20191228_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20191229.nc ../data/era5/mslp/20191229_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20191230.nc ../data/era5/mslp/20191230_mslp.nc
D:\data\ERA5_nc\MSLP\MSLP_20191231.nc ../data/era5/mslp/20191231_mslp.nc
